In [1]:
import pandas as pd
import re
import nltk
from nltk import word_tokenize,sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer 
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.cluster import AgglomerativeClustering
import numpy as np
from sklearn.externals import joblib
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
reviews = pd.read_csv('C:\\WPI\\InfoRet\\FinalProject\\data\\reviews_details.csv',encoding = "ISO-8859-1")
# reviews['listing_id'].value_counts()
# vc = reviews['listing_id'].value_counts()

In [3]:
def remove_nonalphanumeric(text):
    return re.sub("[^a-zA-Z0-9]"," ", str(text)).lower()

In [4]:
unique_listings = reviews.listing_id.unique()
len(unique_listings)

3986

In [5]:
reviews=reviews.groupby('listing_id').filter(lambda g: (g.listing_id.size >= 5))
reviews = reviews[reviews['listing_id'] != 15041925]
reviews = reviews[reviews['listing_id'] !=19102778]

unique_listings = reviews.listing_id.unique()
len(unique_listings)

2821

In [6]:
reviews['comments_1'] = reviews['comments'].apply(remove_nonalphanumeric)
reviews['tokenized_comments'] = reviews['comments_1'].apply(word_tokenize)
stopset = stopwords.words('english') + list(string.punctuation)
reviews['stop_comments'] = reviews['tokenized_comments'].apply(lambda x: [item for item in x if item not in stopset])
def func(row):
    return " ".join(row)
reviews['data'] = reviews['stop_comments'].apply(lambda x: func(x))



In [7]:
no_top_words = 5
no_top_documents = 10
result=[]

In [63]:
results_df2.to_csv("Result_kmeans.csv",columns=["topics","Review_comments","Review_Id","Listing_Id","Documents","Cluster"])

In [9]:
def top_tfidf_feats(row, features, top_n=10):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

In [10]:
def top_mean_feats(Xtr, features, grp_ids=None, min_tfidf=0.1, top_n=25):
    ''' Return the top n features that on average are most important amongst documents in rows
        indentified by indices in grp_ids. '''
    if grp_ids:
        D = Xtr[grp_ids].toarray()
    else:
        D = Xtr.toarray()

    D[D < min_tfidf] = 0
    tfidf_means = np.mean(D, axis=0)
    return top_tfidf_feats(tfidf_means, features, top_n)

In [23]:
#TEST CASE
results_df2 = pd.DataFrame()
for xx in range(0,1):
    listing_reviews=reviews[reviews['listing_id']==14586440]
    tf_vectorizer = TfidfVectorizer(max_df=0.80, min_df=2, max_features=500, stop_words='english',use_idf=True, ngram_range=(1,3))
    clust_tfidf= tf_vectorizer.fit_transform(listing_reviews['data'])
    terms = tf_vectorizer.get_feature_names()
    
    ###Calculate HAC
    num_clusters = 5
    dist = 1 - cosine_similarity(tfidf_matrix)
    linkage_matrix = AgglomerativeClustering(n_clusters=num_clusters) #define the linkage_matrix using ward clustering pre-computed distances
    HAC=linkage_matrix.fit_predict(dist)
    #print(tfidf_matrix)
    #print(HAC)
    #print(len(HAC))
    result=pd.DataFrame({'Review_comments':listing_reviews['comments'],'Review_Id':listing_reviews['id'],"Listing_Id":listing_reviews['listing_id'],"Documents": listing_reviews['data'], "Cluster":HAC })  
    
    

    #FIND MIN INDICES
    z=result['Cluster']
    ind=z.index.tolist()
    minind=min(ind)
    topics=[]
    clust_words=[]
    for j in range(0,5):
        subset=z[z==j]
        groups=subset.index.tolist()
        clust=[]
        for i in groups:
            clust.append(i-minind)
        x=top_mean_feats(clust_tfidf,terms,clust)

        words=[]
        for i in range(0,len(x)):
            words.append(x.iloc[i]['feature'])
        clust_words.append(words)
    topics=[]
    for i in range(0,len(HAC)):
        topics.append(clust_words[HAC[i]])
    print('TOPICS')
    result['topics']=topics
    results_df2 = results_df2.append(result)

   

TOPICS


In [32]:
results_df = pd.DataFrame()
for each_listing in unique_listings:
    print(each_listing)
    listing_reviews=reviews[reviews['listing_id']==each_listing]
    print(listing_reviews.shape)
    tf_vectorizer = TfidfVectorizer(max_df=0.80, min_df=2, max_features=500, stop_words='english',use_idf=True, ngram_range=(1,3))
    clust_tfidf= tf_vectorizer.fit_transform(listing_reviews['data'])
    terms = tf_vectorizer.get_feature_names()
    
    ###Calculate HAC
    num_clusters = 5
    dist = 1 - cosine_similarity(clust_tfidf)

    linkage_matrix = AgglomerativeClustering(n_clusters=num_clusters) #define the linkage_matrix using ward clustering pre-computed distances
    HAC=linkage_matrix.fit_predict(dist)
    #print(tfidf_matrix)
    print(HAC)
    print(len(HAC))
    
    result=pd.DataFrame({'Review_comments':listing_reviews['comments'],'Review_Id':listing_reviews['id'],"Listing_Id":listing_reviews['listing_id'],"Documents": listing_reviews['data'], "Cluster_HAC":HAC })  
    
    #FIND MIN INDICES
    z=result['Cluster_HAC']
    ind=z.index.tolist()
    minind=min(ind)
    topics=[]
    clust_words=[]
    for j in range(0,5):
        subset=z[z==j]
        groups=subset.index.tolist()
        clust=[]
        for i in groups:
            clust.append(i-minind)
        x=top_mean_feats(clust_tfidf,terms,clust)

        words=[]
        for i in range(0,len(x)):
            words.append(x.iloc[i]['feature'])
        clust_words.append(words)
    topics=[]
    for i in range(0,len(HAC)):
        topics.append(clust_words[HAC[i]])
    print('TOPICS')
    result['Topics_HAC']=topics
    results_df = results_df.append(result)
    
print(results_df)
results_df.to_csv("Result_HAC.csv",columns=["Topics_HAC","Review_comments","Review_Id","Listing_Id","Documents","Cluster_HAC"])

7441144
(5, 10)
[3 4 1 2 0]
5
TOPICS
12233830
(75, 10)
[2 2 2 0 0 2 2 2 4 2 2 2 4 2 2 2 2 2 2 2 1 2 0 2 1 1 2 2 2 1 2 4 2 0 0 1 2
 0 0 0 0 2 0 3 1 0 2 1 0 0 3 1 2 2 2 1 2 4 3 1 0 2 0 2 2 4 0 2 2 1 2 0 4 2
 1]
75
TOPICS
14586440
(99, 10)
[4 1 4 4 4 1 4 1 1 1 4 3 4 4 1 0 4 4 1 1 1 4 1 4 4 4 1 4 0 4 1 1 1 1 1 4 0
 1 1 4 2 4 4 0 4 4 0 1 1 1 1 1 4 3 4 1 4 1 4 0 4 1 1 0 0 4 4 1 1 4 4 1 0 1
 0 1 4 4 1 4 1 1 4 4 4 0 2 4 4 2 0 4 4 4 1 0 3 4 4]
99
TOPICS
15444930
(20, 10)
[3 2 0 4 3 0 2 4 4 1 1 1 2 1 0 0 2 3 0 3]
20
TOPICS
1596470
(68, 10)
[1 0 4 2 4 0 0 0 2 0 4 2 0 0 4 2 1 3 1 2 3 3 1 4 0 0 3 2 3 1 4 0 3 4 0 2 2
 0 0 4 2 1 2 2 4 0 3 2 1 0 4 3 2 0 3 3 2 2 1 2 0 2 2 0 2 2 0 4]
68
TOPICS
8211468
(16, 10)
[3 2 3 4 1 1 4 0 3 1 0 1 3 0 0 2]
16
TOPICS
8082512
(10, 10)
[4 4 0 3 1 1 2 3 0 2]
10
TOPICS
20726840
(6, 10)
[0 3 4 0 2 1]
6
TOPICS
17526985
(36, 10)
[0 0 0 2 4 0 0 2 1 0 2 2 3 2 0 1 4 2 3 0 4 0 3 1 1 0 1 2 2 2 2 2 1 2 2 4]
36
TOPICS
13707814
(5, 10)
[2 3 4 1 0]
5
TOPICS
16322051
(21, 10)
[0 3 2 

C:\Users\Claire Danaher\Anaconda3\lib\site-packages\scipy\cluster\hierarchy.py:471: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  return linkage(y, method='ward', metric='euclidean')



990668
(131, 10)
[4 1 1 0 1 1 0 0 0 1 1 0 0 1 1 0 1 1 1 4 1 0 0 3 0 0 2 4 3 4 0 0 0 0 0 0 0
 1 1 4 2 1 0 0 2 4 1 1 1 1 1 0 1 0 3 0 0 0 0 4 0 0 1 1 1 0 0 0 0 0 1 1 1 4
 0 0 0 0 0 3 0 1 0 0 1 3 0 0 0 4 1 1 2 2 0 1 0 0 0 4 1 0 1 0 1 0 1 3 2 1 0
 0 1 3 0 4 0 0 0 1 0 1 0 0 4 0 0 0 0 0 4]
131
TOPICS
19328794
(9, 10)
[4 2 1 0 1 0 2 3 4]
9
TOPICS
1494726
(13, 10)
[2 4 2 3 0 1 4 2 3 1 0 4 0]
13
TOPICS
15507108
(23, 10)
[1 1 2 4 1 2 2 1 2 0 0 2 3 3 2 0 4 3 3 0 0 2 0]
23
TOPICS
13054720
(119, 10)
[0 1 0 0 1 0 0 4 0 0 0 1 0 2 1 1 1 1 1 3 2 0 4 0 0 0 4 1 1 0 1 4 0 1 0 0 1
 1 1 3 3 0 1 0 1 1 1 1 4 1 0 0 0 1 3 0 0 1 4 0 1 0 0 0 4 1 4 1 1 1 0 1 0 4
 1 2 0 2 0 1 1 0 1 4 0 1 0 1 3 4 0 0 1 2 0 1 2 1 2 0 1 4 0 0 1 0 0 2 2 0 0
 1 2 0 2 1 1 3 0]
119
TOPICS
16600533
(12, 10)
[0 1 3 1 2 0 2 2 4 3 2 1]
12
TOPICS
19415743
(16, 10)
[1 1 3 0 1 4 3 1 2 2 2 0 4 0 1 1]
16
TOPICS
639111
(6, 10)
[4 3 1 0 2 0]
6
TOPICS
12210223
(18, 10)
[3 0 2 4 0 0 3 1 3 0 1 0 4 1 2 0 1 0]
18
TOPICS
15390344
(9, 10)
[1 3 1 3 0 0 4 2 

TOPICS
11964926
(84, 10)
[0 0 2 2 2 4 0 2 0 2 0 1 2 0 0 0 1 2 3 1 0 1 0 0 0 1 2 2 0 2 2 1 0 4 1 2 3
 0 1 2 1 3 4 0 1 1 2 0 1 1 2 1 0 4 1 1 0 4 4 4 1 2 4 2 1 0 2 0 0 0 3 0 4 1
 1 2 0 1 2 1 4 0 1 1]
84
TOPICS
5526487
(63, 10)
[0 0 4 2 4 4 0 1 0 1 0 0 1 0 0 1 0 3 1 0 0 0 1 1 4 0 1 0 2 3 1 3 4 1 0 4 0
 1 0 0 3 1 1 3 1 1 4 0 0 0 4 0 0 2 4 3 3 0 1 4 1 0 0]
63
TOPICS
6838488
(11, 10)
[0 4 4 1 3 3 1 0 2 2 0]
11
TOPICS
4645486
(28, 10)
[1 1 1 2 4 0 4 0 0 1 1 0 3 2 3 0 0 0 0 0 2 4 1 0 3 3 1 0]
28
TOPICS
4549374
(124, 10)
[0 4 0 0 1 3 3 3 4 4 1 0 0 1 4 1 0 0 0 0 0 0 0 0 1 0 4 0 0 0 1 2 0 0 1 0 1
 4 0 3 1 0 0 1 0 0 0 3 0 0 0 1 0 0 4 0 4 3 1 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0
 1 3 0 0 0 3 3 0 1 1 1 3 0 1 0 0 3 1 0 1 1 2 0 0 1 0 1 1 3 1 0 3 1 0 1 2 0
 4 4 1 1 0 0 1 0 1 1 0 3 0]
124
TOPICS
19870249
(11, 10)
[2 0 0 3 1 2 4 4 3 1 1]
11
TOPICS
12752442
(63, 10)
[4 1 3 0 4 2 0 2 4 1 3 0 0 4 0 3 0 4 2 0 2 0 2 0 2 4 0 2 4 4 3 4 4 1 0 1 1
 2 0 4 2 2 4 4 4 1 0 3 2 4 4 2 1 2 2 4 2 0 2 4 0 0 3]
63
TOPICS
15232044
(

[3 0 3 3 1 3 0 3 2 4 0 3 3 3 0 1 3 3 0 0 0 0 0 3 3 0 3 0 0 0 0 4 3 0 0 0 1
 3 0 2 3 3 4 1 3 3 3 3 3 4 4 3 1 2 3 4 3 3 3 4 0 4 3 4 3 1 4 4 3 3 4 4 0 3
 3 3 0 2 3 4 0 4 4 3 3 2 1 1 0 3 3 4 4 2 0 3 4 0 2 0 1 0 3 1 0 3 4 3 3 4 0
 2 0 0 0 3 0 0 0 3 3 0 3 4 2 0 3 2 2 0 3 2 3 3 3 4 0 1 3 0 0 4 3 4 4 3 0 0
 0 1 3 3 3 3 0 4 4 3 4 4 1 4 2 0 0 3 2 0 3 4 0 4 0 4 1 0 3 3 3 4 4 3 3 2 2
 0 4 4 4 2 1 0 1 1 1 3 4 3 0 0 4 0 3 1 1 1 0 3 0 0 2 0 1 1 0 3 0 2 3 3 3 3
 0 1 0 4 3 2 3 2 3 3 0 3 0 1 2 3 4 4 0 2 4 0 4 0 1 1 0 2 0 0 0 2 1 0 3 4 4
 4 4 2 3 0 3 0 3 3 0 4 0 0 0 1 0 0 4 4 4 0 3 0 0 0 4 0 0 0 1 1 0 2 3 3 4 2
 4 4 2 0 4 1 0 0 4 1 2 0 0 4 0 0 0 0 4 4 4 1 3 3 0 0 4 4 4 0 3 0 3 0 1 3 0
 1 1 3 0 2 2 4 4 4 3 4 4 3 0 0 0 1 4 4 4 0 3 4 0 4 0 3 0 0 0 1 0 0 4 3 3 4
 4 3 0 3 0 4 0 1]
378
TOPICS
4603803
(13, 10)
[3 3 2 0 2 0 2 2 1 0 0 4 1]
13
TOPICS
18550644
(18, 10)
[2 1 1 4 1 3 3 1 1 1 1 0 0 0 2 0 2 0]
18
TOPICS
7472068
(70, 10)
[0 4 0 0 0 0 0 1 0 0 2 3 2 1 2 2 2 1 1 1 1 4 0 2 2 1 0 0 0 2 2 2 3 3 2 4 2
 0 4 0 3

...

5269216
(66, 10)
[0 1 0 0 1 0 0 1 1 1 0 1 1 4 2 1 1 0 0 0 1 1 2 0 0 0 1 1 1 1 0 3 0 1 1 0 3
 1 1 1 4 4 1 4 4 0 1 1 1 0 4 1 3 4 0 0 4 0 1 1 1 0 1 0 1 1]
66
TOPICS
1115394
(215, 10)
[0 0 1 0 0 4 4 1 0 2 1 1 0 4 1 1 0 0 0 0 1 1 0 0 2 0 0 4 0 1 1 4 1 1 0 1 0
 2 4 1 0 1 4 0 1 3 2 0 0 0 3 1 0 0 0 0 2 4 0 0 1 1 0 4 0 4 1 0 0 0 4 4 1 0
 0 4 0 0 0 0 1 4 0 1 0 1 0 0 0 0 0 0 1 0 0 4 3 2 0 0 0 1 0 2 4 4 3 0 2 0 1
 0 0 2 2 0 0 4 3 4 0 0 0 0 0 1 0 0 3 0 1 1 0 1 4 4 0 0 0 0 1 2 0 4 0 0 4 4
 4 0 0 4 4 1 1 0 0 0 1 4 1 1 3 4 4 0 1 1 4 4 4 1 4 2 3 2 4 4 0 4 1 1 1 1 0
 1 0 2 1 1 4 0 4 4 4 4 1 3 4 4 3 1 4 4 0 3 4 4 0 1 4 0 0 1 1]
215
TOPICS
15173372
(25, 10)
[2 1 1 1 3 3 0 4 3 3 1 3 0 0 4 0 0 1 2 4 1 1 2 1 0]
25
TOPICS
6870485
(7, 10)
[0 4 2 3 0 1 1]
7
TOPICS
17920249
(5, 10)
[2 3 4 1 0]
5
TOPICS
18448985
(35, 10)
[4 1 0 1 2 4 1 0 0 3 1 1 1 2 4 0 3 3 1 3 0 1 4 3 2 4 4 2 4 2 2 4 0 0 3]
35
TOPICS
5726250
(24, 10)
[4 3 0 0 3 0 0 1 1 0 4 1 1 1 2 0 2 1 1 2 4 0 2 2]
24
TOPICS
8231382
(80, 10)
[4 0 1 1 3 1 3 1 4 

TOPICS
3755609
(35, 10)
[4 4 1 1 1 1 3 0 1 0 1 0 2 2 1 0 2 0 0 0 0 1 0 3 0 3 2 1 2 3 1 3 2 2 1]
35
TOPICS
18485290
(29, 10)
[2 1 0 2 0 4 0 0 1 1 4 3 3 4 3 0 0 1 4 4 0 0 3 0 1 2 1 1 0]
29
TOPICS
15148487
(41, 10)
[3 1 1 2 1 1 1 4 1 0 0 1 2 2 1 0 4 1 0 3 2 0 0 0 4 0 1 0 1 0 2 4 2 0 0 0 2
 1 0 2 2]
41
TOPICS
19199046
(45, 10)
[0 1 0 0 0 1 4 3 0 3 1 1 3 0 0 1 0 0 3 3 1 4 1 2 3 3 1 1 3 1 3 2 0 3 1 1 1
 1 0 3 1 1 0 1 0]
45
TOPICS
5592866
(16, 10)
[1 3 1 3 4 2 1 1 2 4 3 1 2 3 0 0]
16
TOPICS
197727
(30, 10)
[1 3 4 3 0 1 3 0 4 4 0 2 4 4 2 1 2 1 0 4 0 1 4 0 2 2 1 2 4 0]
30
TOPICS
2372459
(9, 10)
[3 0 0 1 0 1 2 4 3]
9
TOPICS
14575184
(36, 10)
[1 0 1 3 0 0 1 0 0 4 1 0 3 0 0 1 1 0 2 2 0 0 3 4 2 1 0 0 1 0 2 2 0 0 1 4]
36
TOPICS
10070404
(35, 10)
[4 4 0 0 1 1 0 3 0 3 3 1 1 0 2 0 0 2 0 0 3 0 1 1 0 0 0 3 1 2 1 0 0 0 0]
35
TOPICS
9072821
(5, 10)
[3 4 2 1 0]
5
TOPICS
18193553
(7, 10)
[0 2 1 4 0 3 1]
7
TOPICS
13652639
(10, 10)
[4 1 2 3 2 1 0 3 4 0]
10
TOPICS
17296595
(10, 10)
[0 1 1 0 1 2 4 0 3 2]
10
TOPI

TOPICS
2216767
(70, 10)
[4 4 0 2 0 2 1 0 2 1 1 2 0 2 2 1 2 2 1 3 1 0 0 4 4 1 4 2 1 2 4 1 4 0 1 3 0
 2 4 2 1 4 4 2 4 4 2 2 2 4 0 0 4 4 3 1 1 4 2 2 2 0 1 1 0 0 3 0 1 4]
70
TOPICS
1009275
(27, 10)
[4 2 4 2 3 0 2 1 4 3 1 2 1 2 2 1 4 1 0 0 2 1 2 1 2 1 0]
27
TOPICS
19545605
(5, 10)
[3 4 1 2 0]
5
TOPICS
13586128
(10, 10)
[0 1 2 4 0 3 1 3 0 2]
10
TOPICS
8309711
(86, 10)
[0 1 1 0 0 0 0 0 3 1 3 4 2 0 0 0 0 0 1 1 1 2 1 0 4 1 1 2 1 1 0 1 4 0 1 1 1
 0 1 0 0 1 1 0 1 0 0 4 0 1 3 0 1 0 0 0 1 1 1 1 0 4 1 0 1 1 0 4 1 0 0 4 0 1
 0 1 1 4 1 0 1 4 0 1 1 0]
86
TOPICS
16459191
(42, 10)
[1 1 1 1 4 3 3 3 1 1 2 3 1 0 3 3 4 0 0 2 1 3 3 1 1 4 1 2 0 3 4 3 1 1 1 0 0
 1 0 0 4 3]
42
TOPICS
6065674
(18, 10)
[2 0 0 1 0 3 0 3 4 1 2 4 0 1 0 0 1 2]
18
TOPICS
20392363
(10, 10)
[1 0 3 1 2 4 3 4 2 0]
10
TOPICS
9676551
(174, 10)
[0 0 0 0 0 0 0 4 2 0 1 0 0 0 0 0 1 0 0 0 0 0 0 4 0 0 0 0 0 0 2 1 0 0 0 3 0
 1 0 0 0 0 0 0 0 0 0 2 2 0 0 0 0 0 0 0 3 0 2 2 3 0 0 0 0 0 0 0 1 1 0 0 2 0
 0 0 0 2 0 0 0 0 0 0 0 2 0 4 0 0 0 0 0 0 0 0 0 0 0 

7474855
(15, 10)
[3 4 0 2 0 0 1 2 1 0 1 1 3 4 2]
15
TOPICS
14083611
(156, 10)
[4 0 0 0 0 2 2 4 4 3 1 3 1 1 2 0 1 2 4 1 3 1 1 2 1 0 0 1 4 4 4 0 0 3 2 0 3
 4 2 1 1 0 2 2 4 0 4 2 0 4 2 4 0 1 0 1 4 2 0 2 2 2 4 2 0 2 1 4 3 0 0 1 0 2
 4 4 0 4 1 0 3 0 0 3 4 4 2 2 0 3 4 2 2 4 0 1 1 4 3 4 4 1 0 1 4 3 1 2 4 0 3
 1 0 2 3 4 4 2 2 2 1 0 2 4 4 2 3 2 3 0 4 3 4 2 0 0 1 0 0 4 3 1 1 2 1 4 1 1
 2 2 2 0 0 0 0 1]
156
TOPICS
12962923
(7, 10)
[0 4 2 2 0 3 1]
7
TOPICS
17710012
(26, 10)
[2 4 2 0 0 3 4 0 3 1 3 0 3 1 0 2 4 2 2 2 0 4 2 1 3 2]
26
TOPICS
16904896
(5, 10)
[2 4 3 1 0]
5
TOPICS
2701124
(51, 10)
[0 2 1 0 0 0 3 4 2 4 0 0 0 1 1 0 2 0 0 0 2 0 2 3 3 4 0 0 2 2 2 4 2 4 2 1 0
 0 1 3 4 2 4 0 1 1 0 1 3 1 2]
51
TOPICS
12143408
(35, 10)
[0 0 0 4 0 1 1 0 3 3 0 4 3 1 0 1 1 1 0 2 2 1 1 0 0 0 0 2 1 0 0 0 3 0 0]
35
TOPICS
18377950
(5, 10)
[3 2 4 1 0]
5
TOPICS
11589644
(29, 10)
[2 1 1 2 2 1 2 3 2 1 0 0 3 1 2 1 0 1 1 0 0 2 0 2 2 0 0 4 4]
29
TOPICS
4687499
(24, 10)
[2 3 1 2 0 0 1 0 0 2 2 1 4 0 0 3 1 0 2 0 1 0 4 0]
24
TOP

14566386
(38, 10)
[1 0 0 0 0 1 4 1 0 0 1 0 0 0 0 0 0 2 1 0 1 2 0 0 2 3 0 3 0 2 0 1 4 0 0 4 3
 0]
38
TOPICS
16540349
(34, 10)
[0 0 3 1 4 2 0 0 1 0 2 2 4 2 2 2 2 2 2 2 1 0 3 0 2 2 1 3 1 1 2 0 1 2]
34
TOPICS
20711007
(6, 10)
[2 0 3 4 1 0]
6
TOPICS
18823105
(12, 10)
[1 2 4 4 3 3 2 1 2 0 0 0]
12
TOPICS
19384244
(6, 10)
[0 2 4 3 1 0]
6
TOPICS
18575841
(28, 10)
[3 0 4 1 2 3 3 1 1 1 1 1 2 1 0 0 0 1 4 0 0 1 3 4 0 1 3 2]
28
TOPICS
10004575
(14, 10)
[3 3 3 0 1 1 0 4 2 2 2 1 0 0]
14
TOPICS
14994629
(78, 10)
[0 3 1 3 1 3 3 1 3 0 0 0 3 3 1 3 3 1 3 3 0 1 3 0 3 3 2 3 1 0 1 1 1 3 1 3 1
 1 3 0 3 3 1 1 1 3 1 1 3 3 4 1 3 1 3 3 3 3 3 1 3 1 3 1 3 4 1 2 3 1 0 0 1 1
 1 1 3 3]
78
TOPICS
11710680
(14, 10)
[4 3 1 0 2 2 1 0 2 4 0 4 1 0]
14
TOPICS
4699828
(58, 10)
[0 0 2 0 0 0 0 4 0 4 0 0 0 0 3 0 0 1 2 1 1 1 1 1 1 1 1 1 4 0 0 0 0 0 2 2 0
 2 2 0 0 2 0 0 0 2 0 0 0 0 0 0 0 0 2 0 0 3]
58
TOPICS
1340290
(9, 10)
[0 2 1 2 0 3 1 4 2]
9
TOPICS
8780440
(17, 10)
[2 1 1 2 2 3 3 4 0 0 0 0 0 0 1 4 3]
17
TOPICS
8227206
(26, 10)


In [31]:
results_df.to_csv("Result_HAC.csv",columns=["Topics_HAC","Review_comments","Review_Id","Listing_Id","Documents","Cluster_HAC"])